In [2]:
from tensorflow.keras.models import Sequential  # type: ignore
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten # type: ignore
from tensorflow.keras.optimizers import Adam  # type: ignore
from tensorflow.keras.preprocessing.image import ImageDataGenerator # type: ignore


In [3]:
train_data_gen=ImageDataGenerator(rescale=1./255)
validation_data_gen=ImageDataGenerator(rescale=1./255)


In [4]:
# preprocessing train images
train_generator=train_data_gen.flow_from_directory(
     'datasets/train',   # label all images inside angry as angry and with all 
     target_size=(48,48),
     batch_size=64,
     color_mode="grayscale",
     class_mode='categorical'
)


# preprocessing test images
validation_generator=validation_data_gen.flow_from_directory(
     'datasets/test',   
     target_size=(48,48),
     batch_size=64,
     color_mode="grayscale",
     class_mode='categorical'
)


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [11]:
emotion_model=Sequential()
emotion_model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(48,48,1)))
emotion_model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024,activation='relu'))
emotion_model.add(Dropout(0.25))
emotion_model.add(Dense(7,activation='softmax'))

emotion_model.compile(loss='categorical_crossentropy',optimizer=Adam(learning_rate=0.0001),metrics=['accuracy'])



c:\Users\kushr\OneDrive\Desktop\Emotion Detection Project\emotiondetect\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
from tensorflow.keras.callbacks import EarlyStopping  # type: ignore

early_stopping = EarlyStopping(monitor='val_loss', patience=5)

emotion_model_info = emotion_model.fit(
    train_generator,
    steps_per_epoch=28709 // 64,
    epochs=70,
    validation_data=validation_generator,
    validation_steps=7178 // 64,
    callbacks=[early_stopping]
)



Epoch 1/20
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 422ms/step - accuracy: 0.3097 - loss: 1.7272

c:\Users\kushr\OneDrive\Desktop\Emotion Detection Project\emotiondetect\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


448/448 ━━━━━━━━━━━━━━━━━━━━ 258s 575ms/step - accuracy: 0.3097 - loss: 1.7271 - val_accuracy: 0.3864 - val_loss: 1.5848
Epoch 2/20
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 354us/step - accuracy: 0.3281 - loss: 1.5684 - val_accuracy: 0.5000 - val_loss: 1.5907
Epoch 3/20


C:\Users\kushr\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


448/448 ━━━━━━━━━━━━━━━━━━━━ 106s 237ms/step - accuracy: 0.3974 - loss: 1.5720 - val_accuracy: 0.4339 - val_loss: 1.4818
Epoch 4/20
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 104us/step - accuracy: 0.3594 - loss: 1.6302 - val_accuracy: 0.4000 - val_loss: 1.4014
Epoch 5/20
448/448 ━━━━━━━━━━━━━━━━━━━━ 99s 220ms/step - accuracy: 0.4291 - loss: 1.4856 - val_accuracy: 0.4669 - val_loss: 1.4046
Epoch 6/20
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 89us/step - accuracy: 0.4062 - loss: 1.5487 - val_accuracy: 0.3000 - val_loss: 1.7514
Epoch 7/20
448/448 ━━━━━━━━━━━━━━━━━━━━ 87s 194ms/step - accuracy: 0.4670 - loss: 1.4031 - val_accuracy: 0.4831 - val_loss: 1.3466
Epoch 8/20
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 64us/step - accuracy: 0.4688 - loss: 1.2319 - val_accuracy: 0.6000 - val_loss: 1.1768
Epoch 9/20
448/448 ━━━━━━━━━━━━━━━━━━━━ 99s 221ms/step - accuracy: 0.4934 - loss: 1.3413 - val_accuracy: 0.5059 - val_loss: 1.3048
Epoch 10/20
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 54us/step - accuracy: 0.4219 - loss: 1.5256 - val_accu

 This above code is run on another device because of high processing power hence the .json and weights file is different from the mentioned below and its accuracy is higher.

In [15]:
model_json=emotion_model.to_json()
with open("emotion_model.json",'w') as json_file:
     json_file.write(model_json)

In [17]:
emotion_model.save_weights('emotion_model.weights.h5')